In [ ]:
import numpy as np
import random as rand

# This assumes you have already seen the usual schematics and
# diagrams that show up first when you google "perceptron" and
# "neural network"

## My first perceptron

In [ ]:
# I have this input set, consisting of three values
input = [1, 0, 1]

# and want this output
target = 1

# We have one perceptron node, with one incoming connection per
# input value, where the incoming connections have some
# random weights
node = [0.1, 0.2, -0.05]

In [ ]:
# When a perceptron is activated, it calculates an output value
# by multiplying each input value by the weight of its 
# incoming connection, then summing them all up....
def sum_prod(a_node, some_input):
    the_sum = 0.0
    for weight_index in range(len(a_node)):
        the_sum += a_node[weight_index] * some_input[weight_index]
    return the_sum;

# ...then passing the resulting number through an activation function.
# There are several possibilities for such a function, let's use
# the most common one here: the sigmoid function
def sigm(raw_val): 
    return 1.0/(1.0 + np.exp(-raw_val)) # sigmoid

In [ ]:
# Let's generate our first ever perceptron output!
output = sigm(sum_prod(node, input))
print(output)

In [ ]:
# Hmm... not quite the output we wanted. Let's look at the error
# 1/2 * sum_over_outputs(output - target)^2:
error = 0.5 * (output - target) ** 2
print(error)

## Backpropagation

In [ ]:
#### Some math. Skip if you want. #########
# Let's see how much we should adjust the 3 weights. We will
# differentiate the error function for each weight w_i to know 
# which way to adjust the weights so that the error gets smaller.
# I) expand d(0.5 *(output - target)^2)/dw_i to
#    d(0.5 *(sigm(sum_prod(ws,inputs)) - target)^2)/dw_i
# II) Use chain rule on d(...)^2/dw_i --> 2(...) * d(...)/dw_i:
#    (sigm(sum_prod(ws,inputs)) - target) 
#           * d(sigm(sum_prod(ws,inputs)) - target)/dw_i
# III) "target" in the second term is a constant and disappears
#    (sigm(sum_prod(ws,inputs)) - target) 
#           * d(sigm(sum_prod(ws,inputs)))/dw_i
# IV) Use chain rule on d(sigm(...))/dw_i --> d(sigm())/w_i * d(...)/dw_i
#     This results in the derivative of the activation function and
#     another term:
#    (sigm(sum_prod(ws,inputs)) - target) 
#           * output * (1- output)
#           * d(sum_prod(ws,inputs))/dw_i
# V) A nice boon is that all the terms of the error sum (except the 
#    one containing the weight we deal with) are treated as constants
#    and becoming zeroes, disappear from our derivative:
#    d(i_1*w_1 + i_2*w_2 + i_3*w_3)/dw_2 = 0 + i_2 + 0 = i_2
#
#    (sigm(sum_prod(ws,inputs)) - target) 
#           * output * (1- output)
#           * input_i
# VI) We already know the value of sigm(sum_prod(ws,inputs))
#     In the formula -- it's our output value. So our derivate is:
# 
#    (output - target) * (output * (1- output)) * input_i
#### Continue here. #######################

In [ ]:
# So let's get to tweaking our weights. By what amount should
# we adjust them and in what direction.
# The answer is: by the slope of our error 0.5 *(output - target)^2
# Lets' calculate that in code.
# First step, what's the outer function derived?
diff = output - target

In [ ]:
# But this is not the whole picture: When calculating the derivative
# of the error 1/2*(output - target)^2, we need to apply the chain
# rule, multiplying "diff" with the derivative of output (because
# output also depends on the weights).
# Where did "output" come from? That's right, we need to get the
# derivative of the activation function:
def sigm_der(output):
    return output * (1- output)

In [ ]:
# Putting the last two steps together, we get an important function
# of backpropagation:
def delta(output, target):
    return (output - target) * sigm_der(output)

In [ ]:
# The last step of getting the adjustments for each individual weight is 
# multiplying what we have so for with the derivative of the multiplication
# of each input value with their corresponding weight, d(sumProd)/dw_i.
# Deriving input_i * weight_i for weight_i, yields, unsurprisingly, the input_i.
# Now multiply this value with each weight's input, and we get how much
# we should adjust that weight:
def adjusted_weights(a_node, node_inputs, output, target):
    node_copy = a_node[:]
    for weight_index in range(len(node_copy)):
        slope_climb = delta(output, target) * node_inputs[weight_index]
        slope_descent_step = - (0.1 * slope_climb)
        node_copy[weight_index] += slope_descent_step
    return node_copy

In [ ]:
# Now learn, my pretty, learn!
node = adjusted_weights(node, input, output, target)

In [ ]:
# Now calculate the perceptron's output again with the adjusted weights!
output2 = sigm(sum_prod(node, input))
print(str(output2) + " is closer to " + str(target) + " than " + str(output))

## Application: logical AND

In [ ]:
# Great! Have you already learned something? Because your perceptron has!

In [ ]:
# Now let's apply this over and over, with not just one set of inputs,
# but some more data:
inputs = [
 [1, 0, 1],
 [0, 1, 1],
 [1, 1, 1],
 [0, 0, 1]
]

# We want to output 1 when our first and second input value are both 1,
# 0 otherwise (logical AND)
targets_AND = [0, 0, 1, 0]

# I'm lazy, so let's use the same "random" initial weights again
initial_node = [0.1, 0.2, -0.05]

In [ ]:
# Let 'er rip
def train(num_epochs, a_node, some_inputs, some_targets):
    for epoch_idx in range(num_epochs):
        err_sum = 0.0
        for input_idx in range(len(some_inputs)):
            curr_input = inputs[input_idx]
            output = sigm(sum_prod(a_node, curr_input))
            a_node = adjusted_weights(a_node, curr_input, output, some_targets[input_idx])
            err_sum += 0.5 * (output - some_targets[input_idx]) ** 2
        if (epoch_idx % 200 == 0):
            print("Iteration "+str(epoch_idx)+" Error: "+str(err_sum / 100))
            err_sum = 0.0
    return a_node

trained_node_AND = train(2000, initial_node, inputs, targets_AND)

In [ ]:
# Of course we're now using the error on the same data that we train on...
# But hey, live dangerously!

# Let's try some values to test:
print(sigm(sum_prod(trained_node_AND, [1,0,1]))) # Zero-ish, kinda
print(sigm(sum_prod(trained_node_AND, [1,1,1]))) # One-ish
print(sigm(sum_prod(trained_node_AND, [0,0,1]))) # Even more zero


In [ ]:
# We can of make the node learn much faster, e.g. by tuning the
# learning rate and/or starting with a higher learning rate which
# decreases with the number of iterations (simulated annealing),
# or using more ideas such as momentum.

## Application: logical OR

In [ ]:
# We can also learn other stuff like logical OR (we can re-use the same inputs):
targets_OR = [1, 1, 1, 0]

trained_node_OR = train(2000, initial_node, inputs, targets_OR)

print(sigm(sum_prod(trained_node_OR, [1,0,1]))) # One-ish, kinda
print(sigm(sum_prod(trained_node_OR, [0,1,1]))) # Same
print(sigm(sum_prod(trained_node_OR, [0,0,1]))) # Somewhat zero

## Bias

In [ ]:
# Ok, now you now exactly how a perceptron works.
# So where to go from here?

# First, you should know why there was always that third input value
# lurking around, always being 1. It's called a bias. 
# Without it, the output value 0 (or all-zero input values) could get
# us stuck (the descent would be 0, too).
# Nodes have a weight for the bias, just like for any other input. You can
# imagine it a bit like the intercept in a linear function.

# In most cases, you provide every layer in a multilayer perceptron
# with a bias. We'll do so from here on.

## XOR Fail & Moar neurons

In [ ]:
# So we've got a one-neuron perceptron, which is somewhat like a neural net
# already, right? Let's make it play Go!

# Just kidding, let's try it on logical XOR (exclusive OR) first
# and give it more iterations to make sure it learns well:

targets_XOR = [1, 1, 0, 0]

trained_node_XOR = train(6000, initial_node, inputs, targets_XOR)

print(sigm(sum_prod(trained_node_XOR, [1,0,1]))) # Hmmm, should be 1
print(sigm(sum_prod(trained_node_XOR, [0,1,1]))) # Same here
print(sigm(sum_prod(trained_node_XOR, [0,0,1]))) # Should have been 0
print(sigm(sum_prod(trained_node_XOR, [1,1,1]))) # Same, hard to say


In [ ]:
# So we trained a completely useless perceptron that always returs 1/2
# instead of 0 or 1.
# I'd call that an outspokenly indecisive perceptron (TM).

# Of course you guessed it: We need more power = more neurons.

# The most common neural networks are called multilayer perceptrons
# and are organized as fully-connected feed-forward networks.
# This means that they have M distinctive layers of 1-N perceptrons each
# and the output of every node of layer 1 acts as input of each of the 
# neurons in the following layer 2, and so on.

In [ ]:
# Let's build a simple 2-layer network (we'll use numpy to
# create truly random nodes).
# ...
# ....Also, for good measure, let's pass the activation function
# and it's derivative as well, so the layer knows them and doesn't have
# to rely on too many globally defined functions....
def create_layer(num_nodes, num_inputs, uses_bias, act_func, act_der_func, learningrate):
    num_weights = num_inputs + (1 if uses_bias else 0)
    nodes = 2*np.random.random((num_nodes, num_weights))-1 # *2-1 to center around 0
    #nodes = np.random.random((num_nodes, num_weights))/num_nodes + 0.001 # 0.0001-0.01
    return (nodes, act_func, act_der_func, learningrate)

layer1 = create_layer(num_nodes=2, num_inputs=2, uses_bias=True, act_func=sigm, act_der_func=sigm_der, learningrate=0.1)
# results in something like
# [[ 0.35335893, -0.52181256,  0.20123706],  # first node with weights
#  [-0.93498546,  0.31766613, -0.15244034]] # second node with weights

layer2 = create_layer(num_nodes=1, num_inputs=2, uses_bias=True, act_func=sigm, act_der_func=sigm_der, learningrate=0.1)
# results in something like
# [[-0.32673713,  0.3158073 ,  0.78059177]] # only one node in this layer

In [ ]:
# We'll create outputs per layer with a function that can handle a whole
# layer at a time and an input set and returns the output of all the nodes
# of that layer.
def layer_output(a_layer, input_set):
    nodes = a_layer[0]
    act_func = a_layer[1]
    uses_bias = len(nodes[0]) == len(input_set) + 1
    biased_inputs = np.append(input_set, 1) if uses_bias else input_set
    sumprods = np.dot(nodes, biased_inputs) # np.dot() = loop of sum_prod()
    # results in something like array([ 2.01576287,  1.51741144])
    # 2 nodes, 2 sumprods, two outputs (numpy applies the function to every 
    # sumprod automatically):
    return act_func(sumprods)

In [ ]:
# Let's test if it works:
print(layer_output(layer1, [1,0]))
# [ 0.63520124  0.25210333]

In [ ]:
# Now make it neat and put the network into one variable
network = [
    create_layer(num_nodes=2, num_inputs=2, uses_bias=True, act_func=sigm, act_der_func=sigm_der, learningrate=0.1), # layer 1 (input layer)
    create_layer(num_nodes=1, num_inputs=2, uses_bias=True, act_func=sigm, act_der_func=sigm_der, learningrate=0.1) # layer 2 (output layer)  
]

In [ ]:
# A function to loop over the layers and feed output forward from one layer
# to the next.
def network_output(a_network, input_set):
    curr_data = input_set
    for curr_layer in a_network:
        curr_data = layer_output(curr_layer, curr_data)
    return curr_data

print(network_output(network, [1,0]))
# Results in something like
# [ 0.40050464] # output in a list because the last layer also can have several nodes

In [ ]:
# We've already got some pretty generic (read: powerful) functions here. Now let's
# also make multilayer networks learn. Recall that we calculated the weight
# adjustments for one single perceptron with the formula:

#     (output - target) * act_der(output) * input_i * (-0.1)

# This still applies for our last layer. But what about the previous layer,
# which has an output, but doesn't have an explicit target (we only have
# a defined target for the last layer).
# Well, "output - target" means how much our output is off, and as it turns
# out, one layer's output is the next layer's input. And when our next layer's
# input weight's have to be adjusted up or down, THIS is actually our layer's
# "off-by-x" amount.

# So, in our formula for non-output layers, we replace "output - target" by
# the "act_der(output) * input_i" of the next layer (I'll call this "diff"):

#     next_layers_diff * act_der(output) * input_i * (-0.1)


In [ ]:
# Let's create a function to adjust the weights of all the nodes in a layer
# (it still uses inputs and outputs, but not for calculating the diffs):
def adjust_layer(a_layer, some_inputs, outputs, output_diffs):
    nodes = a_layer[0]
    act_dev_func = a_layer[2]
    learningrate = a_layer[3]
    # Remember: inputs, outputs and output_diffs are 1D-numpy arrays.
    # nodes is a 2D numpy array (rows = nodes, cols = weights)
    uses_bias = len(nodes[0]) == len(some_inputs) + 1
    biased_inputs = np.append(some_inputs, 1) if uses_bias else some_inputs
    # do what our delta function did previously, only for all nodes in one step:
    deltas = output_diffs * act_dev_func(outputs)
    
    # [d1, d2] --> [[d1],   necessary for np.dot product
    #               [d2]]
    deltas_transposed = np.expand_dims(deltas, axis=1)
    
    # all deltas (one for each output/node) multiplied by all inputs:
    gradient_climb = deltas_transposed.dot([biased_inputs]) # dot = loop of sumProds
    gradient_descent_step = -learningrate * gradient_climb
    # [[w1, w2, w3], # node 1 weight changes
    #  [w1, w2, w3]] # node 2 weight changes
    
    # gradient_descent_step has the same dimensions as our layer, so we can just
    # add the steps to the weights in one operation:
    nodes += gradient_descent_step
    
    # Return the weight "off-bys". These will be our "next_layers_diff":
    return deltas.dot(nodes[:,0:len(some_inputs)])

In [ ]:
# Let's try it.
print("Uncorrected weights of output layer:")
print(layer2[0])

# Adjust weights for all nodes in that layer once:
adjust_layer(layer2, [1, 0], np.asarray([0.9]), [ 0.9 - 0 ])

print("Result of adjustment:")
print(layer2[0])

In [ ]:
# Now package this into a neat little function to loop over all the layers
# and propagate errors back from layer to layer.
# In order to avoid having to transfer a lot of inputs/outputs between functions,
# here we will first apply every layer forward, then do backpropagation with the
# results.
def adjust_network(a_network, input_set, targets):
    # forward run to obtain necessary inputs/outputs, layer by layer
    input_per_layer = []
    output_per_layer = []
    curr_data = input_set
    for curr_layer in a_network:
        input_per_layer.append(curr_data)
        curr_data = layer_output(curr_layer, curr_data)
        output_per_layer.append(curr_data)
    # backward run with backprop:
    # start with calculating "diff" a.k.a. "off-by" of output layer:
    last_outputs = output_per_layer[len(a_network)-1]
    output_diffs = last_outputs - targets
    for layer_index in reversed(range(len(a_network))):
        #print("layer "+str(layer_index))
        #print("input: ")
        #print(input_per_layer[layer_index])
        #print("output")
        #print(output_per_layer[layer_index])
        #print("output-diffs")
        #print(output_diffs)
        curr_layer = a_network[layer_index]
        curr_inputs = input_per_layer[layer_index]
        curr_outputs = output_per_layer[layer_index]
        output_diffs = adjust_layer(curr_layer, curr_inputs, curr_outputs, output_diffs)
    # Let's return our actual network output so have something to look at
    return last_outputs

In [ ]:
# Ok, let's see if this works:
# Original network
print(network)
# Do 1 learning iteration with some hard-coded values:
adjust_network(network, [1, 0], [0])
# Changed network:
print(network)

In [ ]:
# A new train function for multilayer networks
def train(num_epochs, a_network, some_inputs, some_targets):
    for epoch_idx in range(num_epochs):
        err_sum = 0.0
        for input_idx in range(len(some_inputs)):
            curr_input = some_inputs[input_idx]
            curr_target = some_targets[input_idx]
            output = adjust_network(a_network, curr_input, curr_target)
            err_sum += np.mean(0.5 * (output - curr_target) ** 2)
            iteration = epoch_idx*len(some_inputs) + input_idx
            if (iteration % 1000 == 0):
                print("Epoch "+str(epoch_idx)+", Iteration "+str(iteration)+" Error: "+str(err_sum / 100))
                err_sum = 0.0

In [ ]:
# Let's try again with XOR:
inputs = [
 [1, 0],
 [0, 1],
 [1, 1],
 [0, 0]
]
targets_XOR = [1, 1, 0, 0]

network = [
    create_layer(num_nodes=3, num_inputs=2, uses_bias=True, act_func=sigm, act_der_func=sigm_der, learningrate=0.1), # layer 1 (input layer - we need 3 nodes here)
    create_layer(num_nodes=1, num_inputs=3, uses_bias=True, act_func=sigm, act_der_func=sigm_der, learningrate=0.1) # layer 2 (output layer)  
]

train(6000, network, inputs, targets_XOR)

print(network_output(network, [1,0])) # 1-ish
print(network_output(network, [0,1])) # At least somewhat closer to 1 than to 0
print(network_output(network, [1,1])) # Goes into the 0 direction, looks about right
print(network_output(network, [0,0])) # Zero. Close enough.


In [ ]:
# Ok, nice...
# But was all that necessary just for XOR? -- Of course not.
# Can it play Go now? -- Still not very well, but it can do some
# other nifty things already.

In [ ]:
# MNIST Digit Handwriting Recognition
def read_mnist(file_name):
    mnist_train = np.genfromtxt(file_name, delimiter=',')
    # get inputs (pixels) and scale them to be between (somewhat) 0 and (nearly) 1
    mnist_inputs = mnist_train[:,1:] / (255.0 * 1.005) + 0.001
    # get outputs (numbers) as 0...9 one-hot vectors
    mnist_outnums = np.int_(mnist_train[:,0])
    mnist_targets = np.zeros((len(mnist_train), 10))
    mnist_targets[np.arange(len(mnist_train)), mnist_outnums] = 1.0
    return (mnist_inputs, mnist_targets)

def print_bitmap(pixels, width):
    levels = ".:+I#"
    line = ""
    for p in pixels:
        lev_idx = int(p * len(levels))
        #print(str(int(p)) + ", "+str(lev_idx))
        line += levels[lev_idx]
        if (len(line) % width == 0):
            print(line)
            line = ""

mnist_inputs, mnist_targets = read_mnist('mnist_train_10000.csv')

In [ ]:
# The rest is just the same as before, just with a few more neurons
mnist_network = [
    create_layer(num_nodes=200, num_inputs=784, uses_bias=True, act_func=sigm, act_der_func=sigm_der, learningrate=0.1), # layer 1 (input layer - one input per pixel)
    create_layer(num_nodes=10, num_inputs=200, uses_bias=True, act_func=sigm, act_der_func=sigm_der, learningrate=0.1) # layer 2 (output layer)  
]

train(15, mnist_network, mnist_inputs, mnist_targets)

In [ ]:
# Now let's test it
mnist_test_inputs, mnist_test_targets = read_mnist('mnist_test_1000.csv')

def test_mnist(network):
    num_correct = 0.0
    for idx in range(len(mnist_test_inputs)):
        out_vec = np.around(network_output(network, mnist_test_inputs[idx]))
        out_num = np.int_(out_vec.dot(np.arange(10)))
        targ_num = np.int_(mnist_test_targets[idx].dot(np.arange(10)))
        num_correct += 1.0 if out_num == targ_num else 0.0
        if (rand.randint(0,50) == 0):
            result = "correct" if out_num == targ_num else "WRONG  "
            print(result + ": "+str(out_num)+" vs. "+str(targ_num))
            if (out_num != targ_num):
                print_bitmap(mnist_test_inputs[idx], 28)
    acc = num_correct/len(mnist_test_inputs)
    print(str(int(acc*100))+"% correctly recognized.")

test_mnist(mnist_network)

In [ ]:
# Now, it reaches some 80-something percent accuracy pretty fast
# and eventually flattens out at about 88%

# We can add changes which lead to significant improvements in accuracy,
# of which many are quite different, such as adding convolutional layers and pooling
# layers (leaving the fully-connected paradigm).

In [ ]:
# We can still experiment with simply adding more layers, though, and see what
# happens:

mnist_network2 = [
    create_layer(num_nodes=200, num_inputs=784, uses_bias=True, act_func=sigm, act_der_func=sigm_der, learningrate=0.1), # layer 1 (input layer - one input per pixel)
    create_layer(num_nodes=100, num_inputs=200, uses_bias=True, act_func=sigm, act_der_func=sigm_der, learningrate=0.1), # layer 2 (hidden layer)
    create_layer(num_nodes=10, num_inputs=100, uses_bias=True, act_func=sigm, act_der_func=sigm_der, learningrate=0.1) # layer 3 (output layer)  
]

train(15, mnist_network2, mnist_inputs, mnist_targets)

test_mnist(mnist_network2)